In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from pyswarms.single import GlobalBestPSO
torch.manual_seed(420)


In [2]:
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [3]:
data

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [5]:
data.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [6]:
col = data.columns
col =[ 'Age', 'Experience', 'Income', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard','Personal Loan']
data = data[col]

In [7]:
data.duplicated().sum()

13

In [8]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
x.shape,y.shape

((5000, 11), (5000,))

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=69)
(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

((4000, 11), (1000, 11), (4000,), (1000,))

In [10]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# x_train = sc.fit_transform(x_train)

In [11]:
X_train = torch.from_numpy(x_train).to(torch.float32)
Y_train = torch.from_numpy(y_train)

### DATALOADER ITERATOR

In [12]:
df = TensorDataset(X_train,Y_train)
data_load = DataLoader(df,batch_size=5)

### Multilayer perceptron model

In [13]:
class NeuralNet(nn.Module):
    def __init__(self,x,h,y):
        super().__init__()
        self.l1 = nn.Linear(x,h)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(h,y)
    def forward(self,val):
        val = self.l1(val)
        val = self.relu(val)
        val = self.l2(val)
        return val    

In [14]:
#input_size
x = X_train.shape[1]
# hidden layer size
h = X_train.shape[1] #most preferably the size of input
#output size
y = 2
mod = NeuralNet(x,h,y)
x,y,h

(11, 2, 11)

In [15]:
x = data_load.dataset

### Loss and optimizing

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(mod.parameters(),lr=0.003)


In [17]:
epoch = 50
pr=0
training_loss = [0] * epoch
training_accuracy = [0] * epoch
for i in range (epoch):
    
    for x1,y1 in data_load:
        #forward
        output = mod(x1)
        loss = criterion(output,y1)
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_loss[i] += loss.item() * y1.size(0)
        correct_count = (torch.argmax(output, dim=1) == y1).to(torch.float32)
        training_accuracy[i] += correct_count.mean() * y1.size(0)
    training_loss[i] /= len(data_load.dataset)
    training_accuracy[i] /= len(data_load.dataset)
    
    pr = loss.item()    
    print("Epoch:",i+1,"Error:",training_loss[i],training_accuracy[i])    

Epoch: 1 Error: 0.5479701197245503 tensor(0.8622)
Epoch: 2 Error: 0.28283180166532473 tensor(0.8907)
Epoch: 3 Error: 0.23164258688177142 tensor(0.9062)
Epoch: 4 Error: 0.2029285317253016 tensor(0.9172)
Epoch: 5 Error: 0.18086702852727285 tensor(0.9258)
Epoch: 6 Error: 0.16406502360923242 tensor(0.9342)
Epoch: 7 Error: 0.15610560146188618 tensor(0.9392)
Epoch: 8 Error: 0.1453615463767676 tensor(0.9417)
Epoch: 9 Error: 0.1372090900799094 tensor(0.9463)
Epoch: 10 Error: 0.12911531624443684 tensor(0.9482)
Epoch: 11 Error: 0.12626023604373812 tensor(0.9535)
Epoch: 12 Error: 0.11986383121337298 tensor(0.9550)
Epoch: 13 Error: 0.11435514340375391 tensor(0.9580)
Epoch: 14 Error: 0.11158135324729301 tensor(0.9580)
Epoch: 15 Error: 0.1089985324519256 tensor(0.9595)
Epoch: 16 Error: 0.1080066664444297 tensor(0.9597)
Epoch: 17 Error: 0.104412053792189 tensor(0.9615)
Epoch: 18 Error: 0.10347149923556742 tensor(0.9617)
Epoch: 19 Error: 0.10334819684267074 tensor(0.9595)
Epoch: 20 Error: 0.1015878369

In [18]:
def flattening(model):
    matrix=[]
    for parameters in model.parameters():
        matrix.append(parameters.numpy().flatten())
         
    return np.concatenate(matrix) 

In [19]:
def shape_size(model):
    shape=[]
    for i in model.parameters():
        shape+=[i.numpy().shape]
    size=[]
    for i in shape:
        if(len(i)==2):
            size+=[i[0]*i[1]]
        else:
            size+=[i[0]]
    return shape,size       

In [32]:
def reshape(child,shape,size):
    param = []
    offset = 0
    for i in range(len(shape)):
        parameter = child[offset : offset + size[i]].reshape(shape[i])
        param.append(parameter)
        offset+=size[i]
    return param

In [23]:
torch.set_grad_enabled(False)
param = flattening(mod)
shape,size=shape_size(mod)


(156, [(11, 11), (11,), (2, 11), (2,)], [121, 11, 22, 2])

In [37]:
options = {'c1':0.6,'c2':0.3,'w':0.1}
x_max = 1.0*np.ones(len(param))
x_min = -1.0*x_max
bounds = (x_min,x_max)

In [30]:
# Accuracy 
def fitness_function(model):
    
    output=model(X_train)
   
    correct_count = (torch.argmax(output, dim=1) == Y_train).to(torch.float32)
    #print(correct_count.sum())
    training_accuracy = correct_count.sum().item()/len(data_load.dataset)
    return training_accuracy

In [63]:
def objective_function(particle,shape=shape,size=size):
    output = []
    acc = []
    
    for i in particle:
        output.append(reshape(i,shape,size))
    output = np.array(output, dtype="object")    
    
    for i in range(len(output)):
        
        for index,weight in enumerate(mod.parameters()):
            weight.data = (torch.tensor(output[i][index])).to(torch.float32)
    return fitness_function(mod)    

In [64]:
pso = GlobalBestPSO(n_particles=500,dimensions=len(param),options=options,bounds=bounds)

In [65]:
best_cost,best_parameters = pso.optimize(objective_func=objective_function,iters=50)
print(1-best_cost)

2023-04-05 21:53:57,966 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████|50/50, best_cost=0.097
2023-04-05 21:54:00,979 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.097, best pos: [-0.62968748  0.62413872 -0.30350696 -0.07230615 -0.80708461  0.79315876
 -0.00712642 -0.63508483  0.07251778  0.39986354 -0.35008944 -0.47033761
  0.70946099  0.77631753  0.25146591 -0.59943521  0.09310169 -0.69645703
 -0.48915891  0.46814833  0.9509025   0.24417977 -0.05584132 -0.56997317
 -0.70151657 -0.18846916 -0.34445816  0.76027501  0.84373498 -0.41278596
 -0.50556442  0.54871676 -0.35158997  0.18799352 -0.32494777  0.89410303
 -0.05401867 -0.41508475  0.16613226 -0.28851128  0.18370312  0.08886279
  0.93714012 -0.95851964  0.11713272  0.22506401 -0.57412408 -0.21856123
 -0.63551124  0.51242633  0.65099683 -0.16701521 -0.609120

0.903


In [67]:
import PyACO.